In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

**LOADING DATASET INTO DATAFRAMES**

---



In [2]:
customers = pd.read_csv('Customers.csv')
customers.head()

,CustomerID,CustomerName,Region,SignupDate
0,C0001,Lawrence Carroll,South America,2022-07-10
1,C0002,Elizabeth Lutz,Asia,2022-02-13
2,C0003,Michael Rivera,South America,2024-03-07
3,C0004,Kathleen Rodriguez,South America,2022-10-09
4,C0005,Laura Weber,Asia,2022-08-15


In [3]:
products=pd.read_csv('Products.csv')
products.head()

,ProductID,ProductName,Category,Price
0,P001,ActiveWear Biography,Books,169.30
1,P002,ActiveWear Smartwatch,Electronics,346.30
2,P003,ComfortLiving Biography,Books,44.12
3,P004,BookWorld Rug,Home Decor,95.69
4,P005,TechPro T-Shirt,Clothing,429.31


In [4]:
transactions=pd.read_csv("Transactions.csv")
transactions.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68


**CHECKING SHAPE**

---



In [5]:
print(customers.shape)
print(products.shape)
print(transactions.shape)

(200, 4)
(100, 4)
(1000, 7)


**CHECKING NULL VALUES**

---



In [6]:
print(customers.isnull().sum())
print(products.isnull().sum())
print(transactions.isnull().sum())

CustomerID      0
CustomerName    0
Region          0
SignupDate      0
dtype: int64
ProductID      0
ProductName    0
Category       0
Price          0
dtype: int64
TransactionID      0
CustomerID         0
ProductID          0
TransactionDate    0
Quantity           0
TotalValue         0
Price              0
dtype: int64


**Exploring Products Dataset**

In [7]:
fig = px.box(products, x='Category', y='Price', title='Price Distribution by Category', points='all', color='Category')
fig.show()

In [8]:
top_products_by_category = products.groupby('Category').apply(lambda x: x.nlargest(3, 'Price')).reset_index(drop=True)

fig = px.bar(
    top_products_by_category,
    x='Category',
    y='Price',
    color='ProductName',
    title='Top 3 Products by Price in Each Category',
    labels={'Price': 'Price (USD)', 'Category': 'Product Category'},
    text='ProductName',
)

fig.update_traces(textposition='outside')
fig.update_layout(
    xaxis_title='Category',
    yaxis_title='Price (USD)',
    showlegend=True,
    legend_title='Product Name',
    barmode='group'
)

fig.show()

<ipython-input-8-71d2fcc3a6f4>:1: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



**Exploring Customers Dataset**

In [9]:
region_counts = customers['Region'].value_counts().reset_index()
region_counts.columns = ['Region', 'CustomerCount']

fig = px.bar(region_counts, x='Region', y='CustomerCount', title='Customer Distribution by Region',range_y=[40,60] )

fig.show()

In [10]:
customers['SignupDate'] = pd.to_datetime(customers['SignupDate'])

monthly_signups = customers.resample('M', on='SignupDate').size().reset_index()
monthly_signups.columns = ['Month', 'SignupCount']

fig = px.line(monthly_signups, x='Month', y='SignupCount', title='Monthly Customer Signups')
fig.show()

<ipython-input-10-83db75c10299>:3: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [11]:
# Oldest customers
oldest_customers = customers.sort_values(by='SignupDate').head(5)
print("Oldest Customers:")
print(oldest_customers)

# Newest customers
newest_customers = customers.sort_values(by='SignupDate', ascending=False).head(5)
print("Newest Customers:")
print(newest_customers)

Oldest Customers:
    CustomerID    CustomerName         Region SignupDate
117      C0118      Jacob Holt  South America 2022-01-22
78       C0079   Brian Murillo  North America 2022-02-02
103      C0104   Laura Bennett  South America 2022-02-10
1        C0002  Elizabeth Lutz           Asia 2022-02-13
165      C0166     John Rogers         Europe 2022-02-13
Newest Customers:
    CustomerID       CustomerName         Region SignupDate
181      C0182     Joshua Preston         Europe 2024-12-28
116      C0117    Jeffrey Mcmahon  North America 2024-11-22
150      C0151     Amber Gonzalez  South America 2024-11-22
113      C0114  Benjamin Anderson         Europe 2024-11-18
65       C0066    Catherine White         Europe 2024-11-15


**Exploring Transactions dataset**

In [12]:
fig = px.histogram(transactions, x='TotalValue', nbins=50, title='Distribution of TotalValue')
fig.show()

In [13]:
# Group by ProductID and sum TotalValue
top_products = transactions.groupby('ProductID')['TotalValue'].sum().reset_index().sort_values(by='TotalValue', ascending=False).head(10)

# Bar plot for Top Products
fig = px.bar(top_products, x='ProductID', y='TotalValue', title='Top Products by TotalValue')
fig.show()

**Combined Exploration of Datasets**

In [31]:
transactions['TransactionDate'] = pd.to_datetime(transactions['TransactionDate'])

transactions['Month'] = transactions['TransactionDate'].dt.to_period('M')

merged_df = pd.merge(transactions, products, on='ProductID', how='left')

merged_df = pd.merge(merged_df, customers, on='CustomerID', how='left')

merged_df

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,Month,ProductName,Category,Price_y,CustomerName,Region,SignupDate
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,2024-08,ComfortLiving Bluetooth Speaker,Electronics,300.68,Andrea Jenkins,Europe,2022-12-03
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,2024-05,ComfortLiving Bluetooth Speaker,Electronics,300.68,Brittany Harvey,Asia,2024-09-04
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,2024-04,ComfortLiving Bluetooth Speaker,Electronics,300.68,Kathryn Stevens,Europe,2024-04-04
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,2024-03,ComfortLiving Bluetooth Speaker,Electronics,300.68,Travis Campbell,South America,2024-04-11
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,2024-03,ComfortLiving Bluetooth Speaker,Electronics,300.68,Timothy Perez,Europe,2022-03-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,T00496,C0118,P037,2024-10-24 08:30:27,1,459.86,459.86,2024-10,SoundWave Smartwatch,Electronics,459.86,Jacob Holt,South America,2022-01-22
996,T00759,C0059,P037,2024-06-04 02:15:24,3,1379.58,459.86,2024-06,SoundWave Smartwatch,Electronics,459.86,Mrs. Kimberly Wright,North America,2024-04-07
997,T00922,C0018,P037,2024-04-05 13:05:32,4,1839.44,459.86,2024-04,SoundWave Smartwatch,Electronics,459.86,Tyler Haynes,North America,2024-09-21
998,T00959,C0115,P037,2024-09-29 10:16:02,2,919.72,459.86,2024-09,SoundWave Smartwatch,Electronics,459.86,Joshua Hamilton,Asia,2024-11-11


In [32]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   TransactionID    1000 non-null   object        
 1   CustomerID       1000 non-null   object        
 2   ProductID        1000 non-null   object        
 3   TransactionDate  1000 non-null   datetime64[ns]
 4   Quantity         1000 non-null   int64         
 5   TotalValue       1000 non-null   float64       
 6   Price_x          1000 non-null   float64       
 7   Month            1000 non-null   period[M]     
 8   ProductName      1000 non-null   object        
 9   Category         1000 non-null   object        
 10  Price_y          1000 non-null   float64       
 11  CustomerName     1000 non-null   object        
 12  Region           1000 non-null   object        
 13  SignupDate       1000 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(3), int64(1

In [33]:
merged_df.isnull().sum()

,0
TransactionID,0
CustomerID,0
ProductID,0
TransactionDate,0
Quantity,0
TotalValue,0
Price_x,0
Month,0
ProductName,0
Category,0


In [34]:
merged_df.describe()

,TransactionDate,Quantity,TotalValue,Price_x,Price_y,SignupDate
count,1000,1000.000000,1000.000000,1000.00000,1000.00000,1000
mean,2024-06-23 15:33:02.768999936,2.537000,689.995560,272.55407,272.55407,2023-07-09 02:49:55.199999744
min,2023-12-30 15:29:12,1.000000,16.080000,16.08000,16.08000,2022-01-22 00:00:00
25%,2024-03-25 22:05:34.500000,2.000000,295.295000,147.95000,147.95000,2022-09-17 12:00:00
50%,2024-06-26 17:21:52.500000,3.000000,588.880000,299.93000,299.93000,2023-07-23 00:00:00
75%,2024-09-19 14:19:57,4.000000,1011.660000,404.40000,404.40000,2024-04-12 00:00:00
max,2024-12-28 11:00:00,4.000000,1991.040000,497.76000,497.76000,2024-12-28 00:00:00
std,NaN,1.117981,493.144478,140.73639,140.73639,NaN


In [35]:
category_sales = merged_df.groupby('Category')['TotalValue'].count().reset_index()

fig = px.bar(category_sales, x='Category', y='TotalValue', title='Total Sales by Product Category')
fig.show()

In [55]:
category_sales = merged_df.groupby('Category')['TotalValue'].sum().reset_index()

fig = px.bar(category_sales, x='Category', y='TotalValue', title='Total Sales by Product Category',range_y=[150_000,200_000])
fig.show()

In [37]:
region_sales = merged_df.groupby('Region')['TotalValue'].sum().reset_index()

fig = px.bar(region_sales, x='Region', y='TotalValue', title='Total Sales by Region',range_y=[150_000,220_000])
fig.show()

In [41]:
merged_df['Month'] = merged_df['Month'].dt.to_timestamp()

monthly_sales = merged_df.groupby('Month')['TotalValue'].sum().reset_index()

fig = px.line(monthly_sales, x='Month', y='TotalValue', title='Monthly Sales Trend')
fig.show()

In [54]:
merged_df['Month'] = pd.to_datetime(merged_df['Month'])

filtered_df = merged_df[merged_df['Month'].dt.month.isin([1, 7, 9])]
category_monthly_sales = filtered_df.groupby(['Category', 'Month'])['TotalValue'].sum().reset_index()

category_monthly_sales['Month'] = category_monthly_sales['Month'].dt.strftime('%B')

fig = px.bar(category_monthly_sales, x='Category', y='TotalValue', color='Month',
             title='Sales by Category for January, July, and September',
             labels={'TotalValue': 'Total Sales', 'Category': 'Product Category'},
             barmode='group')

fig.update_layout(
    xaxis_title="Product Category",
    yaxis_title="Total Sales",
    template="plotly_white"
)

fig.show()

In [43]:
top_customers = merged_df.groupby('CustomerID')['TotalValue'].sum().reset_index().sort_values(by='TotalValue', ascending=False).head(10)

top_customers = pd.merge(top_customers, customers, on='CustomerID', how='left')

fig = px.bar(top_customers, x='CustomerName', y='TotalValue', title='Top 10 Customers by Total Spending')
fig.show()

In [51]:
# Histogram for Product Prices
fig = px.histogram(merged_df, x='Price_x', nbins=10, title='Distribution of Product Prices', labels={'Price_x': 'Price'})
fig.show()

In [53]:
top_products = merged_df.groupby('ProductID')['TotalValue'].sum().reset_index()

top_products = top_products.sort_values(by='TotalValue', ascending=False).head(10)

top_products = pd.merge(top_products, products[['ProductID', 'ProductName']], on='ProductID', how='left')

fig = px.bar(top_products, x='ProductName', y='TotalValue', title='Top 10 Products by Total Sales',
             labels={'ProductName': 'Product', 'TotalValue': 'Total Sales'})

fig.update_layout(
    xaxis_title="Product",
    yaxis_title="Total Sales",
    template="plotly_white"
)

fig.show()

In [59]:
merged_df.describe()

,TransactionDate,Quantity,TotalValue,Price_x,Month,Price_y,SignupDate
count,1000,1000.000000,1000.000000,1000.00000,1000,1000.00000,1000
mean,2024-06-23 15:33:02.768999936,2.537000,689.995560,272.55407,2024-06-08 09:30:14.400000,272.55407,2023-07-09 02:49:55.199999744
min,2023-12-30 15:29:12,1.000000,16.080000,16.08000,2023-12-01 00:00:00,16.08000,2022-01-22 00:00:00
25%,2024-03-25 22:05:34.500000,2.000000,295.295000,147.95000,2024-03-01 00:00:00,147.95000,2022-09-17 12:00:00
50%,2024-06-26 17:21:52.500000,3.000000,588.880000,299.93000,2024-06-01 00:00:00,299.93000,2023-07-23 00:00:00
75%,2024-09-19 14:19:57,4.000000,1011.660000,404.40000,2024-09-01 00:00:00,404.40000,2024-04-12 00:00:00
max,2024-12-28 11:00:00,4.000000,1991.040000,497.76000,2024-12-01 00:00:00,497.76000,2024-12-28 00:00:00
std,NaN,1.117981,493.144478,140.73639,NaN,140.73639,NaN


In [72]:
category_sales = merged_df.groupby('Category')['TotalValue'].sum().reset_index()

# Create a pie chart for product category performance
fig = px.pie(category_sales, values='TotalValue', names='Category', title='Product Category Performance')
fig.show()

In [79]:
purchase_behavior = merged_df['Quantity'].value_counts().reset_index()
purchase_behavior.columns = ['Quantity', 'Count']

fig = px.histogram(purchase_behavior, x='Quantity', y='Count', title='Customer Purchase Behavior',nbins=3)
fig.show()

In [91]:
merged_df['SignupDate'] = pd.to_datetime(merged_df['SignupDate'])
merged_df['TransactionDate'] = pd.to_datetime(merged_df['TransactionDate'])

merged_df['SignupMonth'] = merged_df['SignupDate'].dt.to_period('M')
merged_df['TransactionMonth'] = merged_df['TransactionDate'].dt.to_period('M')
merged_df = merged_df[merged_df['SignupDate'] >= '2024-01-01']

signups_per_month = merged_df.groupby('SignupMonth').size().reset_index(name='Signups')

purchases_per_month = merged_df.groupby('TransactionMonth')['TotalValue'].sum().reset_index()
purchases_per_month.columns = ['SignupMonth', 'TotalPurchases']
purchases_per_month['TotalPurchases'] = purchases_per_month['TotalPurchases'] / 1000
purchases_per_month.columns = ['SignupMonth', 'TotalPurchases']

monthly_data = signups_per_month.merge(purchases_per_month, on='SignupMonth', how='outer')

monthly_data['SignupMonth'] = monthly_data['SignupMonth'].dt.to_timestamp()

fig = px.bar(monthly_data, x='SignupMonth', y=['Signups', 'TotalPurchases'],
             title='Signups and Purchases per Month',
             labels={'SignupMonth': 'Month', 'value': 'Count/Total Value', 'variable': 'Metric'},
             barmode='group')

fig.update_xaxes(title='Month')
fig.update_yaxes(title='Count/Total Value')
fig.show()

In [92]:
monthly_customers = merged_df.groupby('TransactionMonth')['CustomerID'].nunique().reset_index()
monthly_customers.columns = ['TransactionMonth', 'UniqueCustomers']

repeat_purchases = merged_df[merged_df['CustomerID'].duplicated(keep=False)]
monthly_repeat_customers = repeat_purchases.groupby('TransactionMonth')['CustomerID'].nunique().reset_index()
monthly_repeat_customers.columns = ['TransactionMonth', 'RepeatCustomers']

monthly_data = monthly_customers.merge(monthly_repeat_customers, on='TransactionMonth', how='left')
monthly_data['RetentionRate'] = (monthly_data['RepeatCustomers'] / monthly_data['UniqueCustomers']) * 100

monthly_data['TransactionMonth'] = monthly_data['TransactionMonth'].dt.to_timestamp()


In [93]:

# Plot 1: Customer Retention Rates Over Time
fig1 = px.line(monthly_data, x='TransactionMonth', y='RetentionRate',
               title='Customer Retention Rates Over Time',
               labels={'TransactionMonth': 'Month', 'RetentionRate': 'Retention Rate (%)'})
fig1.show()


In [94]:

# Plot 2: Frequency of Repeat Purchases
repeat_purchase_counts = repeat_purchases['CustomerID'].value_counts().reset_index()
repeat_purchase_counts.columns = ['CustomerID', 'RepeatPurchaseCount']

fig2 = px.histogram(repeat_purchase_counts, x='RepeatPurchaseCount',
                    title='Frequency of Repeat Purchases',
                    labels={'RepeatPurchaseCount': 'Number of Repeat Purchases', 'count': 'Number of Customers'})
fig2.show()